In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sframe
%matplotlib inline

In [4]:
loans = sframe.SFrame('lending-club-data.gl/')

[INFO] sframe.cython.cy_server: SFrame v2.1 started. Logging C:\Users\kinsly\AppData\Local\Temp\sframe_server_1475179516.log.0


In [5]:
loans.column_names()

['id',
 'member_id',
 'loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'term',
 'int_rate',
 'installment',
 'grade',
 'sub_grade',
 'emp_title',
 'emp_length',
 'home_ownership',
 'annual_inc',
 'is_inc_v',
 'issue_d',
 'loan_status',
 'pymnt_plan',
 'url',
 'desc',
 'purpose',
 'title',
 'zip_code',
 'addr_state',
 'dti',
 'delinq_2yrs',
 'earliest_cr_line',
 'inq_last_6mths',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'initial_list_status',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'last_pymnt_d',
 'last_pymnt_amnt',
 'next_pymnt_d',
 'last_credit_pull_d',
 'collections_12_mths_ex_med',
 'mths_since_last_major_derog',
 'policy_code',
 'not_compliant',
 'status',
 'inactive_loans',
 'bad_loans',
 'emp_length_num',
 'grade_num',
 'sub_grade_num',
 'delinq_2yrs_zero',
 'pub_rec

In [6]:
loans['safe_loans'] = loans['bad_loans'].apply(lambda x : +1 if x==0 else -1)
loans = loans.remove_column('bad_loans')

In [7]:
features = ['grade',                     # grade of the loan
            'sub_grade',                 # sub-grade of the loan
            'short_emp',                 # one year or less of employment
            'emp_length_num',            # number of years of employment
            'home_ownership',            # home_ownership status: own, mortgage or rent
            'dti',                       # debt to income ratio
            'purpose',                   # the purpose of the loan
            'term',                      # the term of the loan
            'last_delinq_none',          # has borrower had a delinquincy
            'last_major_derog_none',     # has borrower had 90 day or worse rating
            'revol_util',                # percent of available credit being used
            'total_rec_late_fee',        # total late fees received to day
           ]

target = 'safe_loans'                    # prediction target (y) (+1 means safe, -1 is risky)

# Extract the feature columns and target column
loans = loans[features + [target]]

In [8]:
safe_loans_raw = loans[loans[target] == +1]
risky_loans_raw = loans[loans[target] == -1]
print "Number of safe loans  : %s" % len(safe_loans_raw)
print "Number of risky loans : %s" % len(risky_loans_raw)

Number of safe loans  : 99457
Number of risky loans : 23150


In [9]:
# Since there are fewer risky loans than safe loans, find the ratio of the sizes
# and use that percentage to undersample the safe loans.
percentage = len(risky_loans_raw)/float(len(safe_loans_raw))

risky_loans = risky_loans_raw
safe_loans = safe_loans_raw.sample(percentage, seed=1)

# Append the risky_loans with the downsampled version of safe_loans
loans_data = risky_loans.append(safe_loans)


In [10]:
loans_data = risky_loans.append(safe_loans)

categorical_variables = []
for feat_name, feat_type in zip(loans_data.column_names(), loans_data.column_types()):
    if feat_type == str:
        categorical_variables.append(feat_name)

for feature in categorical_variables:
    loans_data_one_hot_encoded = loans_data[feature].apply(lambda x: {x: 1})
    loans_data_unpacked = loans_data_one_hot_encoded.unpack(column_name_prefix=feature)

    # Change None's to 0's
    for column in loans_data_unpacked.column_names():
        loans_data_unpacked[column] = loans_data_unpacked[column].fillna(0)

    loans_data.remove_column(feature)
    loans_data.add_columns(loans_data_unpacked)

In [11]:
train_data, validation_data = loans_data.random_split(.8, seed=1)

9. Now, let's use the built-in scikit learn decision tree learner (sklearn.tree.DecisionTreeClassifier) to create a loan prediction model on the training data. To do this, you will need to import sklearn, sklearn.tree, and numpy.

Note: You will have to first convert the SFrame into a numpy data matrix, and extract the target labels as a numpy array (Hint: you can use the .to_numpy() method call on SFrame to turn SFrames into numpy arrays). See the API for more information. Make sure to set max_depth=6.

Call this model decision_tree_model.

10. Also train a tree using with max_depth=2. Call this model small_model.

In [13]:
import sklearn
import sklearn.tree
features = [x for x in train_data.column_names() if x != "safe_loans"]
train_features = train_data[features].to_numpy()
train_target = train_data['safe_loans'].to_numpy()

In [14]:
decision_tree_model = sklearn.tree.DecisionTreeClassifier(max_depth = 6)
small_model = sklearn.tree.DecisionTreeClassifier(max_depth = 2)

In [36]:
len(train_data[features].column_names())
len(sample_validation_data.column_names())
x = sample_validation_data.column_names()
# x[x == features]
# for i in range(len(x)):
    
#     if x[i] != features[i]:
#         print(x[i])


In [16]:
X = train_features
Y = train_target
decision_tree_model = decision_tree_model.fit(X,Y)
small_model = small_model.fit(X,Y)

In [13]:
dot_file = sklearn.tree.export_graphviz(decision_tree_model.tree_, out_file='tree_d1.dot')  #export the tree to .dot file


In [14]:
# from sklearn.tree import DecisionTreeClassifier
# from sklearn import tree
# from sklearn import datasets

# iris = datasets.load_iris()

# clf = DecisionTreeClassifier(max_depth=1,criterion="entropy") # construct a decision tree.
# clf.fit(iris.data,iris.target)  # train it on the dataset

# dot_file = tree.export_graphviz(clf.tree_, out_file='tree_d1.dot', feature_names=iris.feature_names)  #export the tree to .dot file


In [38]:
validation_safe_loans = validation_data[validation_data[target] == 1]
validation_risky_loans = validation_data[validation_data[target] == -1]

sample_validation_data_risky = validation_risky_loans[0:2]
sample_validation_data_safe = validation_safe_loans[0:2]

sample_validation_data = sample_validation_data_safe.append(sample_validation_data_risky)
sample_validation_data[features]

Columns:
	short_emp	int
	emp_length_num	int
	dti	float
	last_delinq_none	int
	last_major_derog_none	int
	revol_util	float
	total_rec_late_fee	float
	grade.A	int
	grade.B	int
	grade.C	int
	grade.D	int
	grade.E	int
	grade.F	int
	grade.G	int
	sub_grade.A1	int
	sub_grade.A2	int
	sub_grade.A3	int
	sub_grade.A4	int
	sub_grade.A5	int
	sub_grade.B1	int
	sub_grade.B2	int
	sub_grade.B3	int
	sub_grade.B4	int
	sub_grade.B5	int
	sub_grade.C1	int
	sub_grade.C2	int
	sub_grade.C3	int
	sub_grade.C4	int
	sub_grade.C5	int
	sub_grade.D1	int
	sub_grade.D2	int
	sub_grade.D3	int
	sub_grade.D4	int
	sub_grade.D5	int
	sub_grade.E1	int
	sub_grade.E2	int
	sub_grade.E3	int
	sub_grade.E4	int
	sub_grade.E5	int
	sub_grade.F1	int
	sub_grade.F2	int
	sub_grade.F3	int
	sub_grade.F4	int
	sub_grade.F5	int
	sub_grade.G1	int
	sub_grade.G2	int
	sub_grade.G3	int
	sub_grade.G4	int
	sub_grade.G5	int
	home_ownership.MORTGAGE	int
	home_ownership.OTHER	int
	home_ownership.OWN	int
	home_ownership.RENT	int
	purpose.car	int
	purpose.credit_card	int
	purpose.debt_consolidation	int
	purpose.home_improvement	int
	purpose.house	int
	purpose.major_purchase	int
	purpose.medical	int
	purpose.moving	int
	purpose.other	int
	purpose.small_business	int
	purpose.vacation	int
	purpose.wedding	int
	term. 36 months	int
	term. 60 months	int

Rows: 4

Data:
+-----------+----------------+-------+------------------+-----------------------+
| short_emp | emp_length_num |  dti  | last_delinq_none | last_major_derog_none |
+-----------+----------------+-------+------------------+-----------------------+
|     0     |       11       | 11.18 |        1         |           1           |
|     0     |       10       | 16.85 |        1         |           1           |
|     0     |       3        | 13.97 |        0         |           1           |
|     0     |       11       | 16.33 |        1         |           1           |
+-----------+----------------+-------+------------------+-----------------------+
+------------+--------------------+---------+---------+---------+---------+---------+
| revol_util | total_rec_late_fee | grade.A | grade.B | grade.C | grade.D | grade.E |
+------------+--------------------+---------+---------+---------+---------+---------+
|    82.4    |        0.0         |    0    |    1    |    0    |    0    |    0    |
|    96.4    |        0.0         |    0    |    0    |    0    |    1    |    0    |
|    59.5    |        0.0         |    0    |    0    |    0    |    1    |    0    |
|    62.1    |        0.0         |    1    |    0    |    0    |    0    |    0    |
+------------+--------------------+---------+---------+---------+---------+---------+
+---------+---------+--------------+--------------+--------------+--------------+
| grade.F | grade.G | sub_grade.A1 | sub_grade.A2 | sub_grade.A3 | sub_grade.A4 |
+---------+---------+--------------+--------------+--------------+--------------+
|    0    |    0    |      0       |      0       |      0       |      0       |
|    0    |    0    |      0       |      0       |      0       |      0       |
|    0    |    0    |      0       |      0       |      0       |      0       |
|    0    |    0    |      0       |      0       |      0       |      0       |
+---------+---------+--------------+--------------+--------------+--------------+
+--------------+--------------+-----+
| sub_grade.A5 | sub_grade.B1 | ... |
+--------------+--------------+-----+
|      0       |      0       | ... |
|      0       |      0       | ... |
|      0       |      0       | ... |
|      1       |      0       | ... |
+--------------+--------------+-----+
[4 rows x 67 columns]

In [40]:
print decision_tree_model.predict(sample_validation_data[features].to_numpy())
#sample_validation_data.column_names()
print sample_validation_data['safe_loans']

[ 1 -1 -1  1]
[1L, 1L, -1L, -1L]


In [41]:
decision_tree_model.predict_proba(sample_validation_data[features].to_numpy())

array([[ 0.34156543,  0.65843457],
       [ 0.53630646,  0.46369354],
       [ 0.64750958,  0.35249042],
       [ 0.20789474,  0.79210526]])

In [45]:
small_model.predict_proba(sample_validation_data[features].to_numpy())

array([[ 0.41896585,  0.58103415],
       [ 0.59255339,  0.40744661],
       [ 0.59255339,  0.40744661],
       [ 0.23120112,  0.76879888]])

In [44]:
# from IPython.display import Image  
# dot_data = StringIO() 
# sklearn.tree.export_graphviz(small_model, out_file=dot_data,  
#                          feature_names=features,  
#                          class_names=["risky", "safe"],  
#                          filled=True, rounded=True,  
#                          special_characters=True)  
# graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
# graph.write_pdf("iris.pdf")

In [49]:
decision_tree_model.score(train_data[features].to_numpy(),train_data[target].to_numpy())

0.64052761659144641

In [54]:
small_model.score(train_data[features].to_numpy(),train_data[target].to_numpy())

0.61350204169353106

In [55]:
Valid_feature = validation_data[features].to_numpy()
Valid_target = validation_data[target].to_numpy()
decision_tree_model.score(Valid_feature, Valid_target)

0.63614821197759586

In [51]:
big_model = sklearn.tree.DecisionTreeClassifier(max_depth = 10)

big_model = big_model.fit(X,Y)

In [52]:
big_model.score(train_data[features].to_numpy(),train_data[target].to_numpy())

0.66379217709004945

In [56]:
big_model.score(Valid_feature, Valid_target)

0.62645411460577338

In [58]:
predict = decision_tree_model.predict(Valid_feature)

In [67]:
FP = 0
FN = 0
for i in range(len(predict)):
    if Valid_target[i] == -1:
        if predict[i] != Valid_target[i]:
            FP += 1
    else:
        if predict[i] != Valid_target[i]:
            FN += 1
        

In [69]:
print FP*20000
print FN*10000

33220000
17170000


In [71]:
FP*20000 + FN*10000

50390000